# M5 Agentic AI - 市场调研团队

## 1. 引言

### 1.1. 实验概览  

在本实验中，你将扮演一家时尚品牌的**技术 AI 负责人**，筹备夏季太阳镜活动。你的任务是设计一条**全自动的创意流水线**，贴近真实业务场景。与其逐个手动处理，你将引导一个系统：扫描在线来源以发现新兴的时尚趋势，将这些趋势与内部目录中的太阳镜匹配，设计活动视觉，生成简短营销文案，并最终将所有内容打包成一份**可供高管审阅的报告**。  

目标是体验如何将多个代理、工具与模型编排为一个统一、连贯的工作流。完成本实验后，你将构建出一个更像小团队协作解决创意挑战的流水线，而非一串相互孤立的脚本步骤。  

### 1.2. 🎯 学习目标

通过本实验，你将看到如何超越模型的单轮交互，转而设计**多代理流水线**来协同规划、调研与创意生成。你将学习如何用外部工具支撑代理的推理，使输出不仅富有创意，也有真实数据背书。你还会尝试反思与打包步骤，以实施质量控制并为高管受众准备结果。  

简而言之，本实验旨在学习如何将**大语言模型的想象力**与**结构化工作流的纪律性**结合起来，为构建既有创意又可靠的自治系统提供实用范式。  

## 2. 环境设置：导入库并加载环境

与之前的实验一样，现在导入所需库、加载环境变量并设置辅助工具。

In [ ]:
# =========================
# 导入
# =========================

# --- 标准库 ---
import base64
import json
import os
import re
from datetime import datetime
from io import BytesIO

# --- 第三方 ---
import requests
import openai
from PIL import Image
from dotenv import load_dotenv
from IPython.display import Markdown, display
import aisuite

# --- 本地 / 项目 ---
import tools
import utils


# =========================
# 环境与客户端
# =========================
load_dotenv()
client = aisuite.Client()


## 3. 可用工具  

只有当模型在基础推理之外被赋予**明确能力**时，自治流水线才真正奏效。预先声明这些工具可使代理的动作空间清晰明确、让提示自然引导工具选择，并通过定义良好的接口保持编排与测试的透明性。  

你将组建一个**市场调研团队**，由多个专业代理协作设计夏季太阳镜活动。为赋能他们，我们首先定义能以真实数据支撑其推理的工具。  

第一个工具是 `tools.tavily_search_tool`，用于进行实时网页搜索以挖掘当前时尚趋势的证据。现在试试，运行一个简单查询：*“trends in sunglasses fashion”*。  

In [ ]:
tools.tavily_search_tool('trends in sunglasses fashion')

第二个工具是 `tools.product_catalog_tool`，它返回内部的太阳镜目录。每条记录都包含产品名称、ID、描述、库存数量与价格等详情。这些结构化数据使智能体能够将在线时尚趋势与实际库存商品关联起来：

In [ ]:
tools.product_catalog_tool()

有了这些工具，你已定义了清晰的动作空间与可靠的数据来源。下一节将构建使用这些工具的代理，把原始的时尚信号转化为结构化洞见与活动资产。

## 4. 智能体定义 — 组建你的团队

既然工具已就位，现在让它们发挥作用。在这一阶段，你将组建一个**市场调研团队**，由多名专业代理构成，并用自然语言指令进行驱动。  

每个代理都依赖此前引入的工具，协同把原始趋势数据转化为打磨完善的活动报告。我们将逐个定义它们，介绍其角色并展示对应实现代码。  

### 4.1. 市场调研智能体  

借助**市场调研智能体**，你迈出构建活动的第一步。让它使用 `tavily_search_tool` 扫描网络，洞察当前太阳镜时尚的热点趋势；随后用 `product_catalog_tool` 将这些信号与内部目录交叉验证，从而知道哪些产品契合当下。  

该智能体会返回一份简洁的简报：所发现的头部时尚洞见、与之匹配的产品，以及为何这些选择适合你的夏季推广的简短说明。这为后续活动的塑造提供了清晰、数据驱动的基础。  

现在可以运行下方单元，以代码形式定义**市场调研智能体**。  

In [ ]:
def market_research_agent(return_messages: bool = False):

    utils.log_agent_title_html("Market Research Agent", "🕵️‍♂️")

    prompt_ = f"""
你是一名时尚市场调研代理，负责为夏季太阳镜活动准备趋势分析。

目标：
1. 使用网页搜索探索与太阳镜相关的当前时尚趋势。
2. 查看内部产品目录，识别与这些趋势相契合的商品。
3. 从目录中推荐一个或多个最符合新兴趋势的产品。
4. 如需注明，今天的日期是 {datetime.now().strftime("%Y-%m-%d")}。

可调用以下工具：
- tavily_search_tool：发现外部网络趋势。
- product_catalog_tool：检查内部太阳镜目录。

完成分析后，请总结：
- 你发现的 2–3 个主要趋势。
- 与这些趋势匹配的目录产品。
- 为何它们适合夏季活动的理由说明。
"""
    messages = [{"role": "user", "content": prompt_}]
    tools_ = tools.get_available_tools()

    while True:
        response = client.chat.completions.create(
            model="openai:o4-mini",
            messages=messages,
            tools=tools_,
            tool_choice="auto"
        )

        msg = response.choices[0].message

        if msg.content:
            utils.log_final_summary_html(msg.content)
            return (msg.content, messages) if return_messages else msg.content

        if msg.tool_calls:
            for tool_call in msg.tool_calls:
                utils.log_tool_call_html(tool_call.function.name, tool_call.function.arguments)
                result = tools.handle_tool_call(tool_call)
                utils.log_tool_result_html(result)

                messages.append(msg)
                messages.append(tools.create_tool_response_message(tool_call, result))
        else:
            utils.log_unexpected_html()
            return ("[⚠️ Unexpected: No tool_calls or content returned]", messages) if return_messages else "[⚠️ Unexpected: No tool_calls or content returned]"

让我们从**市场调研代理**获取关于夏季太阳镜活动的建议。  

In [ ]:
market_research_result = market_research_agent()

接下来，使用平面设计代理将这份简报转化为视觉概念。

### 4.2. 平面设计智能体  

借助**平面设计智能体**，你将从分析走向创意。  
获取市场调研智能体的简报，并让该智能体将其转化为视觉概念。  
由于 `aisuite` 尚不支持直接图像生成（如 DALL·E），你将以两阶段引导流程：  

1. 首先，代理使用 `aisuite` 与 OpenAI 文本模型（`o4-mini`）生成生动的**提示**与简短、吸引人的**文案**。  
2. 然后，将该提示发送到 OpenAI 的 `dall-e-3` API 以生成**活动图像**本身。  

结果提供了推进所需的一切：生成的图像（本地保存以便复用）、生成它的精确提示（便于迭代）、以及用于活动叙事的精炼文案。  

<div style="border:1px solid #fca5a5; border-left:6px solid #ef4444; background:#fee2e2; border-radius:6px; padding:12px 14px; color:#111827; font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Cantarell,Noto Sans,sans-serif;">
  <strong>注意：</strong> 当前 <code>aisuite</code> <strong>不支持直接图像生成</strong>。  
  因此你将其文本输出（提示 + 文案）与 OpenAI 的 <code>dall-e-3</code> 结合，用于生成最终的活动视觉。
</div>  

现在可以运行下方单元，以代码形式定义**平面设计代理**。  

In [ ]:
def graphic_designer_agent(trend_insights: str, caption_style: str = "short punchy", size: str = "1024x1024") -> dict:

    """
    使用 aisuite 生成营销提示/文案，并直接使用 OpenAI 生成图像。

    参数：
        trend_insights (str)：来自调研智能体的趋势摘要。
        caption_style (str)：文案的可选风格提示。
        size (str)：图像分辨率（例如 '1024x1024'）。

    返回：
        dict：包含 image_url、prompt 与 caption 的字典。
    """

    utils.log_agent_title_html("平面设计智能体", "🎨")

    # 步骤 1: 使用 aisuite 生成提示和文案
    system_message = (
        "你是一名视觉营销助理。根据输入的趋势洞见，"
        "为 AI 图像生成模型编写一个有创意的视觉提示，并生成一段简短文案。"
    )

    user_prompt = f"""
趋势洞见：
{trend_insights}

请输出：
1. 一段生动、具描述性的提示，用于引导图像生成。
2. 一句营销文案，风格：{caption_style}。

按如下格式回应：
{{"prompt": "...", "caption": "..."}}
"""

    chat_response = client.chat.completions.create(
        model="openai:o4-mini",
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt}
        ]
    )

    content = chat_response.choices[0].message.content.strip()
    match = re.search(r'\{.*\}', content, re.DOTALL)
    parsed = json.loads(match.group(0)) if match else {"error": "No JSON returned", "raw": content}

    prompt = parsed["prompt"]
    caption = parsed["caption"]

    # 步骤 2: 直接使用 openai-python 生成图像
    openai_client = openai.OpenAI()

    image_response = openai_client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        size=size,
        quality="standard",
        n=1,
        response_format="url"
    )

    image_url = image_response.data[0].url

    # 本地保存图像
    img_bytes = requests.get(image_url).content
    img = Image.open(BytesIO(img_bytes))

    filename = os.path.basename(image_url.split("?")[0])
    image_path = filename
    img.save(image_path)


    # 使用本地图像记录摘要
    utils.log_final_summary_html(f"""
        <h3>已生成图像与文案</h3>

        <p><strong>图像路径：</strong> <code>{image_path}</code></p>

        <p><strong>生成的图像：</strong></p>
        <img src="{image_path}" alt="Generated Image" style="max-width: 100%; height: auto; border: 1px solid #ccc; border-radius: 8px; margin-top: 10px; margin-bottom: 10px;">

        <p><strong>提示：</strong> {prompt}</p>
    """)


    return {
        "image_url": image_url,
        "prompt": prompt,
        "caption": caption,
        "image_path": image_path  
    }



现在运行 `graphic_designer_agent`，使用**市场调研智能体**提供的趋势洞见生成活动图像。

In [ ]:
graphic_designer_agent_result = graphic_designer_agent(
    trend_insights=market_research_result,
)


拿到视觉素材后，使用文案智能体来打造活动的声音。

### 4.3. 文案智能体  

在**市场调研智能体**与**平面设计智能体**完成其工作后，转向**文案智能体**。当你手头已有活动图像与趋势摘要时，让该智能体为活动创建“声音”。  

它将视觉与分析作为多模态输入，打造一条简短而优雅的营销短句，精准传达核心信息。除了短句，它还提供清晰的理由——为何该短句契合图像，以及它如何与趋势相呼应。  

这样，你不仅获得朗朗上口的文案，也得到其背后的推理，便于在利益相关者面前阐述与优化。  



In [ ]:
def copywriter_agent(image_path: str, trend_summary: str, model: str = "openai:o4-mini") -> dict:

    """
    使用 aisuite（仅 OpenAI）发送图像与趋势摘要并返回活动短句。

    参数：
        image_path (str)：待分析图像的路径。
        trend_summary (str)：来自调研智能体的文本。
        model (str)：OpenAI 模型（例如 openai:o4-mini、openai:gpt-4o）

    返回：
        dict: {
            "quote": "...",
            "justification": "...",
            "image_path": "..."
        }
    """

    utils.log_agent_title_html("文案智能体", "✍️")

    # 步骤 1: 加载本地图像并编码为 base64
    with open(image_path, "rb") as f:
        img_bytes = f.read()

    b64_img = base64.b64encode(img_bytes).decode("utf-8")

    # 步骤 2: 构建兼容 OpenAI 的多模态消息
    messages = [
        {
            "role": "system",
            "content": "你是一名文案撰写者，基于图像与市场趋势摘要创作优雅的活动短句。"
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{b64_img}",
                        "detail": "auto"
                    }
                },
                {
                    "type": "text",
                    "text": f"""
以下为一个视觉营销图像与趋势分析：

趋势摘要：
\"\"\"{trend_summary}\"\"\"

请返回如下 JSON 对象：
{{
  "quote": "简短、优雅的活动短句（最多 12 个词）",
  "justification": "为何该短句契合该图像与趋势"
}}"""
                }
            ]
        }
    ]

    # 步骤 3: 通过 aisuite 发送请求
    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    # 步骤 4: 解析 JSON 响应
    content = response.choices[0].message.content.strip()

    utils.log_final_summary_html(content)

    try:
        match = re.search(r'\{.*\}', content, re.DOTALL)
        parsed = json.loads(match.group(0)) if match else {"error": "No valid JSON returned"}
    except Exception as e:
        parsed = {"error": f"Failed to parse: {e}", "raw": content}


    parsed["image_path"] = image_path
    return parsed


现在调用文案智能体，根据营销图像与先前生成的趋势洞见，产出一条简短的活动短句。

In [ ]:
copywriter_agent_result = copywriter_agent(
    image_path=graphic_designer_agent_result["image_path"],
    trend_summary=market_research_result,
)

当短句与理由准备好后，使用打包智能体将所有内容整合为可供高管审阅的成果。

### 4.4. 打包智能体  

最后，引入**打包智能体**将所有内容串联成整体。在**市场调研智能体**、**平面设计智能体**与**文案智能体**各自完成其部分后，打包智能体会把整个故事整合为一个打磨完善的成果。  

让它接收趋势摘要、活动视觉、生成的短句与理由说明，并将这些内容组装成一份适合高管阅读的 Markdown 报告。过程中，它会重写趋势洞见以提升清晰度与语气、确保短句与图像样式搭配合理，并组织结构使最终文档专业且具说服力。  

完成此步后，你将获得一套完整的活动资料包——易于分享、视觉吸引、并准备好接受高层审阅。  

In [ ]:
def packaging_agent(trend_summary: str, image_url: str, quote: str, justification: str, output_path: str = "campaign_summary.md") -> str:

    """
    将活动资产打包为精美的 Markdown 报告，供高管审阅。

    Args:
        trend_summary (str)：市场趋势摘要。
        image_url (str)：活动图像的 URL。
        quote (str)：需叠加的营销短句。
        justification (str)：短句的理由说明。
        output_path (str)：保存 Markdown 报告的路径。

    Returns:
        str：已保存的 Markdown 文件路径。
    """

    utils.log_agent_title_html("打包智能体", "📦")

    # 我们在 <img> 的 src 中使用此路径
    styled_image_html = f"""
![打开生成的文件查看]({image_url})
    """

    beautified_summary = client.chat.completions.create(
        model="openai:o4-mini",
        messages=[
            {"role": "system", "content": "你是一名市场传播专家，为高管撰写优雅的活动总结。"},
            {"role": "user", "content": f"""
请将以下趋势摘要改写为清晰、专业且适合 CEO 受众的表达：

\"\"\"{trend_summary.strip()}\"\"\"
"""}
        ]
    ).choices[0].message.content.strip()

    utils.log_tool_result_html(beautified_summary)

    # 将所有部分合并为 markdown
    markdown_content = f"""# 🕶️ 夏季太阳镜活动 – 高管摘要

## 📊 精炼的趋势洞见
{beautified_summary}

## 🎯 活动视觉
{styled_image_html}

## ✍️ 活动短句
{quote.strip()}

## ✅ 原因说明
{justification.strip()}

---

*报告生成日期 {datetime.now().strftime('%Y-%m-%d')}*
"""

    with open(output_path, "w", encoding="utf-8") as f:
        f.write(markdown_content)

    return output_path



当趋势摘要、活动图像与短句都已就绪后，将所有内容交给**打包智能体**。它的职责是把这些部分整合为一份精致、可供高管审阅的报告。运行下一个单元来生成它。  


In [ ]:
packaging_agent_result = packaging_agent(
    trend_summary=market_research_result,
    image_url=graphic_designer_agent_result["image_path"],
    quote=copywriter_agent_result["quote"],
    justification=copywriter_agent_result["justification"],
    output_path=f"campaign_summary_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.md"
)

最终结果是一份格式精美的活动报告，你可以在笔记本中直接查看。它将包含：  

- 经精炼的趋势摘要，改写为更适合高管阅读的表述  
- 视觉化样式的图像，并通过 HTML 叠加你的活动短句  
- 清晰的理由说明，帮助你理解为何视觉与信息契合当前趋势  
- 显示报告生成时间的时间戳  

你可以使用如下方式查看：  

In [ ]:
# 加载并渲染 Markdown 内容
with open(packaging_agent_result, "r", encoding="utf-8") as f:
    md_content = f.read()

display(Markdown(md_content))


最后，将整个工作流封装为一个可调用函数，以一步运行完整流水线。

## 5. 完整活动流水线 – `run_sunglasses_campaign_pipeline`

在此步骤中，你将定义一个函数 `run_sunglasses_campaign_pipeline`，把所有部分串联为一个无缝的夏季太阳镜活动工作流。  

该函数将：  
- 运行市场调研，扫描时尚趋势并与目录匹配。  
- 生成具有视觉样式的图像与文案。  
- 创建简短、优雅且带理由说明的活动短句。  
- 将所有内容打包为精美的 Markdown 报告，便于高管审阅。  

定义该函数后，你可以**一次调用运行整个流水线**，同时仍能追踪中间结果并查看最终报告。  

In [ ]:
def run_sunglasses_campaign_pipeline(output_path: str = "campaign_summary.md") -> dict:
    """
    运行完整的夏季太阳镜活动流水线：
    1. 市场调研（搜索趋势并匹配产品）
    2. 生成视觉图像与文案
    3. 基于图像与趋势生成活动短句
    4. 创建高管版 Markdown 报告

    Returns:
        dict: 包含所有中间结果与最终报告路径的字典
    """
    # 1. 运行市场调研Agent
    trend_summary = market_research_agent()
    print("✅ 市场调研完成")

    # 2. 生成图像 + 文案
    visual_result = graphic_designer_agent(trend_insights=trend_summary)
    image_path = visual_result["image_path"]
    print("🖼️ 图像已生成")

    # 3. 基于图像 + 趋势生成短句
    quote_result = copywriter_agent(image_path=image_path, trend_summary=trend_summary)
    quote = quote_result.get("quote", "")
    justification = quote_result.get("justification", "")
    print("💬 短句已生成")

    # 4. 生成 markdown 报告
    md_path = packaging_agent(
        trend_summary=trend_summary,
        image_url=image_path,  
        quote=quote,
        justification=justification,
        output_path=f"campaign_summary_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.md"
    )

    print(f"📦 报告已生成：{md_path}")

    return {
        "trend_summary": trend_summary,
        "visual": visual_result,
        "quote": quote_result,
        "markdown_path": md_path
    }


现在可通过一次调用运行流水线来创建完整的活动报告。执行下一个单元即可：  

In [ ]:
results = run_sunglasses_campaign_pipeline()

### 5.1. 结果

运行下方单元以查看完整活动流水线生成的输出。

In [ ]:
with open(results["markdown_path"], "r", encoding="utf-8") as f:
    md_content = f.read()
display(Markdown(md_content))


## 6. 关键要点  

完成本实验后，你已了解如何：  

- 使用**多代理 LLM 流水线**端到端地自动化创意工作流。  
- 将**推理、工具调用与外部数据**结合，使输出扎根于现实。  
- 应用多模态模型（如 `gpt-4o`）处理**文本与图像**，用于生成活动短句等任务。  
- 通过工具（`tavily_search_tool`、`product_catalog_tool`）扩展模型能力，使输出不仅富有创意，也更务实。  
- 借助结构化日志与 HTML 样式块保持执行**透明且可调试**。  
- 交付格式精美、**适合高管审阅**的 Markdown 报告，将洞见、视觉与理由整合为一个成果。  



<div style="border:1px solid #22c55e; border-left:6px solid #16a34a; background:#dcfce7; border-radius:6px; padding:14px 16px; color:#064e3b; font-family:system-ui,-apple-system,Segoe UI,Roboto,Ubuntu,Cantarell,Noto Sans,sans-serif;">

🎉 <strong>恭喜！</strong> 🎉  

你已成功构建并运行一条**多代理流水线**：完成趋势调研、生成视觉素材、打造活动短句，并把一切打包为**适合高管审阅的报告**。  

该工作流展示了如何将 **LLM 的创造力** 与 **结构化编排的纪律性**结合，为你提供可复用的范式，能够适配多种现实场景。🌟  
</div>
